In [2]:
from datasets import load_dataset

# Load the dataset in streaming mode
ds = load_dataset("tiendung/chai", split="train", streaming=True)

# Filter to only examples with thumbs_up == True
filtered_ds = ds.filter(lambda x: x.get("thumbs_up") is True)

# Preview an example
for example in filtered_ds.take(1):
    print(example)

{'_id': '_bot_5c9283de-131d-4600-9818-11456f7458ca_mk4NsyDQzWSsWjxPYkdBOegmJit2_1689211870273_1689211909', 'conversation': [{'name': 'Barbie Roberts (Super Kind)', 'text': 'PACE!'}, {'name': 'Anonymous user', 'text': 'corn on the cob'}, {'name': 'Barbie Roberts (Super Kind)', 'text': '*laughs* what did you just say?'}], 'languages': ['en'], 'thumbs_up': True, 'feedback': 'corn', 'model_name': 'chaiml-ak-edit-v0_1689207647'}


In [5]:
from datasets import load_dataset

# Load dataset in streaming mode and filter
ds = load_dataset("tiendung/chai", split="train", streaming=True)
filtered_ds = ds.filter(lambda x: x.get("thumbs_up") is True)

def convert_chai_old_format(example):
    convo = example["conversation"]
    if not convo or len(convo) < 3:
        return []

    bot_name = convo[0]["name"]
    bot_intro = convo[0]["text"]

    messages = []

    # First user message includes bot intro
    if convo[1]["name"] != bot_name:
        user_first = convo[1]["text"]
        messages.append({
            "role": "user",
            "content": f"{bot_intro} \n\n {user_first}"
        })

    # First assistant message
    if len(convo) > 2 and convo[2]["name"] == bot_name:
        messages.append({
            "role": "assistant",
            "content": f"{convo[2]['text']}"
        })

    # Rest of the conversation
    for i in range(3, len(convo), 2):
        # User message
        if i < len(convo) and convo[i]["name"] != bot_name:
            messages.append({
                "role": "user",
                "content": f"{convo[i]['text']}"
            })

        # Assistant message
        if i + 1 < len(convo) and convo[i + 1]["name"] == bot_name:
            messages.append({
                "role": "assistant",
                "content": f"{convo[i + 1]['text']}"
            })

    return {"messages": messages}

# # Preview a few examples
# for example in filtered_ds.take(3):
#     formatted = convert_chai_old_format(example)
#     print(formatted)
#     print("=" * 60)

all_examples = []
for example in filtered_ds:
    # text = example
    messages = convert_chai_old_format(example)
    if messages and messages['messages'] != []:  # skip empty ones
        all_examples.append(messages)

In [6]:
import json

with open("../data/combined_data_old_format.jsonl", "w") as f:
    for item in all_examples:
        f.write(json.dumps(item) + "\n")